# Demostration of the recommenders

In [9]:
import pandas as pd
import numpy as np
import time
from surprise import dump
from evaluation.Evaluate import Evaluate, Evaluate_CFR

import model.CollaborativeFilteringRec as CFR
from model.CollaborativeFilteringRec import train_cf_models

#import model.HybridRecommendationSystem as HR
from model.ContentBasedRec import ContentBasedRecommender, get_CBMatrix
from model.HybridRecommendationSystem import HybridRecommender, get_HRMatrix

movie_df = pd.read_csv('./data/external/movies.csv')

# pick a random user
ratings = pd.read_csv('./data/processed/final_ratings.csv')
print(len(ratings.userId.unique()))
random_user = np.random.choice(ratings.userId.unique()) # picks a random user

1004


## Preprocessing
* Preprocessed data for both content-based and colloborative filtering systems.
* Updates the precalculated dataframes.

In [37]:
print("Prerocessing for Collaborative Filtering Recommender...")
from data_script.preprocess_collaborative import sveta_user_id, hamid_user_id
# import clustering_collaborative #TODO

Prerocessing for Collaborative Filtering Recommender...
Creating raw dataframes...
Creating dataframes with Sveta and Hamid ratings...
Creating dataframes with anti-Sveta and anti-Hamid ratings...
Creating new ratings dataframe for collaborative filtering ...
Out of total of  138493  users,  64051  are considered lazy and will be removed.
The final number of users is  74442
Randomly choosing 1000 users...
Adding Sviatlana's and Hamid's ratings... we are lazy users, but still...
Sveta 's user id is 86968
Hamid 's user id is 27844
Adding anti-Sviatlana's and anti-Hamid's ratings... they are lazy users, but still...
anti-Sveta 's user id is 8633
anti-Hamid 's user id is 73687
Now, the final number of users is  1004
Filtering movies...
Out of total of  9486  movies,  8863  are considered rare and will be removed.
The final number of movies is  623
Creating the pivot matrix...
The resulting sparcity of the matrix is: 0.8019191292614454
Preparing the final rating matrix...
Data preprocesssin

In [4]:
#sveta_user_id, hamid_user_id # or use sveta and hamid's id (assigned randomly at the preprocessing step

In [5]:
print("Prerocessing for Content-Based Recommender...")
from data_script.preprocess_content_based import get_df_ContBaseRec
df_ContBaseRec = get_df_ContBaseRec()

Prerocessing for Content-Based Recommender...


/home/mumu/Documents/DS/movie_recommendation_ProjectCDS/data_script/preprocess_content_based.py:93: FutureWarning: Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.sum(level=1) should use df.groupby(level=1).sum().
  dummies = pd.get_dummies(df_movies.genres.apply(pd.Series).stack()).sum(level=0)


## Train Nearest Neighbor CB model

In [2]:
nn = CBR.train_nearest_neighbors_model()

## Training and saving CF models

In [8]:
trainset, testset, data, algo, algo_predictions, knn, knn_predictions = train_cf_models()

Getting the ratings matrix...
Preparing data in the Suprise format...
Training and saving a KNN model for calculating the CF model weights in hybrid setup..
Mean CV RMSE is 0.837
Training KNN-based memory based model for hybrid...
Computing the pearson similarity matrix...
Done computing similarity matrix.
Mean CV RMSE is 0.933
Training is done!


## Content-based recommender

In [ ]:
cb_recommender = ContentBasedRecommender()

In [ ]:
# Run to recomputer the CBMatrix
start_time = time.time()
cb_recommendations = get_CBMatrix()
end_time = time.time()
elapsed_time = end_time - start_time
print("Elapsed time: ", elapsed_time)
cb_recommendations.head()

In [6]:
# Get the CBMatrix
cb_recommender = ContentBasedRecommender()
cb_recommendations = cb_recommender.df_recommendations
cb_recommendations.head()

,userId,1,2,3,4,5,6,7,8,9,...,130075,130087,130490,130496,130520,130578,130840,131013,131168,131170
0,73687,NaN,0.545045,0.665229,NaN,NaN,NaN,NaN,0.558663,0.538763,...,0.519058,NaN,0.449878,0.643524,0.496660,0.533135,0.613908,0.557084,0.649311,0.601671
1,8633,NaN,NaN,0.558319,NaN,NaN,0.807316,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.467845,0.655830,0.605724
2,27844,NaN,0.545045,0.665229,NaN,NaN,NaN,NaN,0.558663,0.538763,...,0.519058,NaN,0.449878,0.643524,0.496660,0.533135,0.613908,0.557084,0.649311,0.601671
3,86968,NaN,NaN,0.558319,NaN,NaN,0.807316,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.467845,0.655830,0.605724
4,186,NaN,0.753088,0.795874,0.730306,0.728662,0.639611,0.724683,0.721785,0.636189,...,0.666016,NaN,0.594067,0.723904,0.659503,0.593578,0.647815,0.630278,0.598122,0.645461


In [3]:
#since the final rating and cb_recommendation matrix do not have the same set of users, we can only test the 
set_of_test_users = set(cb_recommendations.userId.values).intersection(set(ratings.userId.unique()))
print(len(set_of_test_users))

1004


## User-based colloborative filtering recommender

In [11]:
cf_model = CFR.CollaborativeFilteringRecommender()
cf_model.fit_and_predict()
cf_model.recommend(sveta_user_id, 20).merge(movie_df, on='movieId', how='left')

Getting the ratings matrix...
Preparing data in the Suprise format...
Done calculating predictions and scores!


,userId,movieId,rating,cf_score,title,genres
0,62095,32,4.796649,0.954811,Twelve Monkeys (a.k.a. 12 Monkeys) (1995),Mystery|Sci-Fi|Thriller
1,62095,912,4.692394,0.931643,Casablanca (1942),Drama|Romance
2,62095,1230,4.580400,0.906756,Annie Hall (1977),Comedy|Romance
3,62095,1219,4.559330,0.902073,Psycho (1960),Crime|Horror
4,62095,1233,4.499145,0.888699,"Boot, Das (Boat, The) (1981)",Action|Drama|War
5,62095,904,4.461464,0.880325,Rear Window (1954),Mystery|Thriller
6,62095,1204,4.444006,0.876446,Lawrence of Arabia (1962),Adventure|Drama|War
7,62095,5995,4.434802,0.874401,"Pianist, The (2002)",Drama|War
8,62095,1288,4.430102,0.873356,This Is Spinal Tap (1984),Comedy
9,62095,1207,4.423082,0.871796,To Kill a Mockingbird (1962),Drama


## Hybrid recommender

In [4]:
hyb_recommender = HybridRecommender()

In [ ]:
# Run to recomputer the HRMatrix
start_time = time.time()
hyb_recommendations = get_HRMatrix()
end_time = time.time()
elapsed_time = end_time - start_time
print("Elapsed time: ", elapsed_time)
hyb_recommendations.head()

In [5]:
# Get the HRMatrix
hyb_recommendations = hyb_recommender.df_recommendations
hyb_recommendations.head()

,userId,1,2,3,4,5,6,7,8,9,...,130075,130087,130490,130496,130520,130578,130840,131013,131168,131170
0,73687,NaN,0.701052,0.779702,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,8633,NaN,NaN,0.742512,NaN,NaN,0.936585,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,27844,NaN,0.843275,0.871280,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,86968,NaN,NaN,0.771691,NaN,NaN,0.976571,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,186,NaN,0.887033,0.899596,NaN,0.857419,0.849518,0.858104,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
#since the final rating and cb_recommendation matrix do not have the same set of users, we can only test the 
set_of_test_users = set(hyb_recommendations.userId.values).intersection(set(hyb_recommendations.userId.values))
print(len(set_of_test_users))

1000


# Evaluation of the different recomenders


## Accuracy of the collaborative system
* metrics: rmse
* method: cross-validation


In [14]:
cf_model.cross_validate()

Cross Validating the data...
Mean CV RMSE is 0.836


0.836

## Variety
The measure represents the averaged percentage of all the movies recommended to all the users (1 corresponds to 100%).

In [2]:
Evaluate = Evaluate()

Getting the ratings matrix...
Preparing data in the Suprise format...
Done calculating predictions and scores!


In [5]:
variety_cf = Evaluate.variety_collaborative_filtering()
print("Collaborative filtering:", variety_cf)

Collaborative filtering: (0.151, 0.0)


In [3]:
variety_cb = Evaluate.variety_content_based() # as CB is unoptimized, it takes a really long time
print("Content-based:", variety_cb)

Content-based: (0.148, 0.0)


In [3]:
variety_hybrid = Evaluate.variety_hybrid() # as CB is unoptimized, it takes a really long time...
print("Hybrid:", variety_hybrid)

Getting the ratings matrix...
Preparing data in the Suprise format...
Computing the pearson similarity matrix...
Done computing similarity matrix.
Getting the ratings matrix...
Preparing data in the Suprise format...
Computing the pearson similarity matrix...
Done computing similarity matrix.
Getting the ratings matrix...
Preparing data in the Suprise format...
Computing the pearson similarity matrix...
Done computing similarity matrix.
Getting the ratings matrix...
Preparing data in the Suprise format...
Computing the pearson similarity matrix...
Done computing similarity matrix.
Getting the ratings matrix...
Preparing data in the Suprise format...
Computing the pearson similarity matrix...
Done computing similarity matrix.
Getting the ratings matrix...
Preparing data in the Suprise format...
Computing the pearson similarity matrix...
Done computing similarity matrix.
Getting the ratings matrix...
Preparing data in the Suprise format...
Computing the pearson similarity matrix...
Done 

## Personalization
Personalised system is expected to sugest differetn movies for dissimilar users.

Mertic: overlap between suggestions for different users.

Similarity is taken from various similarity matrices.